In [1]:
import torch.utils.data
from torchvision import datasets, models, transforms
import os
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [98]:
## Random blur and random brightness adjustment to be added.
## About normalization we need to check in which values we normalize -> later
data_transforms = transforms.Compose([
        transforms.RandomResizedCrop((640, 512)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])

In [23]:
def generate_out_map(img_name, features, objects_to_look = ['ball'], ratio = 4):
        # ratio is the input-output ratio
        f = features[(features['image_file']==img_name) & (features.label.isin(objects_to_look))]
        out_map = torch.zeros((160, 128))

        for i in range(len(f)):
            temp = f.iloc[i]
            x_object = (temp['xmin'] + temp['xmax'])/(ratio*2) # /2 because you the middle, /4 because of the ratio with original img
            y_object = (temp['ymin'] + temp['ymax'])/(ratio*2) # /2 because you the middle, /4 because of the ratio with original img
            current_value = out_map[x_object.astype(int), y_object.astype(int)]
            if current_value == 0 or current_value < 1:
                out_map[x_object.astype(int), y_object.astype(int)] = 1

        #TODO: how to spread this one as a gaussian from the center, need discussion
        return out_map 


In [24]:
class SoccerBallDetection(Dataset):
    """Soccer Ball dataset."""

    def __init__(self, csv_file, root_dir, transform=None, objects_to_look = ['ball']):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.features = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        # Loading image
        img_name = self.features.iloc[idx, 0]
        img_path = os.path.join(self.root_dir, img_name)
        image = io.imread(img_path)
        
        # Extracting features
        sample = {'image': image, 'features': generate_out_map(img_name, features)}

        if self.transform:
            sample = self.transform(sample)

        return sample
    

In [67]:
features = pd.read_csv('data/data.csv')

In [25]:
dataset = SoccerBallDetection('data/data.csv', 'data/train')

In [56]:
# import numpy as np
# import scipy.stats as st

# def gkern(kernlen=5, nsig=3):
#     """Returns a 2D Gaussian kernel array."""

#     interval = (2*nsig+1.)/(kernlen)
#     x = np.linspace(-nsig-interval/2., nsig+interval/2., kernlen+1)
#     kern1d = np.diff(st.norm.cdf(x))
#     kernel_raw = np.sqrt(np.outer(kern1d, kern1d))
#     kernel = kernel_raw/kernel_raw.sum()
#     return kernel
# gkern(3, 4)

array([[0.01882794, 0.09955906, 0.01882794],
       [0.09955906, 0.52645199, 0.09955906],
       [0.01882794, 0.09955906, 0.01882794]])

In [26]:
dataloader = DataLoader(dataset, batch_size=4,
                        shuffle=True, num_workers=2)

In [27]:
for i_batch, sample_batched in enumerate(dataloader):
    print(i_batch, sample_batched['image'].size(),
          sample_batched['features'].size())
    if(i_batch > 1):
        break

Exception ignored in: <function _DataLoaderIter.__del__ at 0x129b387b8>
Exception ignored in: <function _DataLoaderIter.__del__ at 0x129b387b8>
Traceback (most recent call last):
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 717, in __del__
  File "/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 717, in __del__
    self._shutdown_workers()
  File "/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 713, in _shutdown_workers
    self._shutdown_workers()
    w.join()
  File "/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 713, in _shutdown_workers
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 138, in join
    w.join()
    assert self._parent_pid == os.getpid(), 'can only join a child process'
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 138, in join
AssertionError: can only join a child proc

NameError: Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 138, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 138, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "<ipython-input-24-145da1d44d30>", line 26, in __getitem__
    sample = {'image': image, 'features': generate_out_map(img_name, features)}
NameError: name 'features' is not defined
